In [1]:
!pip install plotly

In [2]:
# Library imports
import panel as pn
pn.extension('plotly')
from panel.interact import interact
import matplotlib.pyplot as plt     
import plotly.tools
import pandas as pd
import hvplot.pandas
import json
import seaborn as sns
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced

In [3]:
# Read in data
df = pd.read_csv('../Data/EFT_optimised_signals.csv', header=[0, 1], parse_dates=True, index_col=[0])

bb_df = pd.read_csv('../Data/BB_optimiser_record.csv', header=[0], index_col=[1])

rf_df = pd.read_csv('../Data/df_random_forrest_results.csv', header=[0], parse_dates=True, index_col=[0])

nb_df = pd.read_csv('../Data/df_naive_bayes_results.csv', header=[0], parse_dates=True, index_col=[0])

nb_j = pd.read_json('../Data/dict_naive_bayes_results.json')

rf_j = pd.read_json('../Data/dict_random_forrest_results.json')

In [4]:
# Make Data easily usable
# Flip DataFrame orientation
rf_df = rf_df.transpose()

# Flip DataFrame orientation
nb_df = nb_df.transpose()

# Convert some columns to int format
nb_df['BackTest % Return'] = pd.to_numeric(nb_df['BackTest % Return'])
nb_df['Buy_Precision'] = pd.to_numeric(nb_df['Buy_Precision'])
nb_df['Sell_Precision'] = pd.to_numeric(nb_df['Sell_Precision'])
nb_df['Sell_Recall'] = pd.to_numeric(nb_df['Sell_Recall'])
nb_df['Buy_Recall'] = pd.to_numeric(nb_df['Buy_Recall'])

# Convert some columns to int format
rf_df['BackTest % Return'] = pd.to_numeric(rf_df['BackTest % Return'])
rf_df['Buy_Precision'] = pd.to_numeric(rf_df['Buy_Precision'])
rf_df['Sell_Precision'] = pd.to_numeric(rf_df['Sell_Precision'])
rf_df['Sell_Recall'] = pd.to_numeric(rf_df['Sell_Recall'])
rf_df['Buy_Recall'] = pd.to_numeric(rf_df['Buy_Recall'])

rf_df['True_Buys %'] =pd.to_numeric(rf_df['True_Buys %'])
rf_df['False_Buys %'] =pd.to_numeric(rf_df['False_Buys %'])
rf_df['True_Sells %'] =pd.to_numeric(rf_df['True_Sells %'])
rf_df['False_Sells %'] =pd.to_numeric(rf_df['False_Sells %'])

In [5]:
# Create a list of all tickers
ticker_list = list(dict.fromkeys((df.droplevel(axis=1, level=[-1]))))

In [6]:
# Ticker dropdown widget
ticker_widget = pn.widgets.Select(options=ticker_list, name='Ticker')

In [26]:
# Plot function for random forrest results
@pn.depends(ticker_widget)
def random_forrest_results(ticker):
    test = rf_df[['Buy_Precision', 'Sell_Precision',	'Buy_Recall', 'Sell_Recall']].loc[ticker].hvplot.bar()
    
    return test

# Plot function for naive bayes results
@pn.depends(ticker_widget)
def naive_bayes_results(ticker):
    test_2 = nb_df[['Buy_Precision', 'Sell_Precision',	'Buy_Recall', 'Sell_Recall']].loc[ticker].hvplot.bar()
    return test_2



# pn.Row(
#      pn.Column(ticker_widget, random_forrest_results),
#      pn.Column(ticker_widget, naive_bayes_results)).servable()

In [8]:
# Function that will return ROI on selected tickers
@pn.depends(ticker_widget) # this will automattically push plot_recent_sentiment_vs_close to update
def backtest_results(ticker):
    rf_backtest = rf_df['BackTest % Return'].loc[ticker]
    
    nb_backtest = nb_df['BackTest % Return'].loc[ticker]
    
    bb_backtest = bb_df['pct_return'].loc[ticker]

    return f'''
    ROI over 10 years using Random Forest Model: {rf_backtest},
    
    ROI over 10 years using Naive Bayes model: {nb_backtest},
    
    ROI over 10 years using Bollinger Bands trading indicator: {bb_backtest}
    '''

# pn.Column(ticker_widget, backtest_results)

In [9]:
>>> from sklearn.metrics import ConfusionMatrixDisplay


In [10]:
rf_j

,XLE,XLF,XLU,XLI,GDX,XLK,XLV,XLY,XLP,XLB,...,GDXJ,IYE,OIH,XME,XRT,SMH,IBB,KBE,KRE,XTL
importances,"{'MACD_FAST_PERIOD': 0.217678806961934, 'RSI':...","{'RSI': 0.193096472635217, 'MACD_FAST_PERIOD':...","{'RSI': 0.144841567361834, 'MACD_FAST_PERIOD':...","{'RSI': 0.219151644704134, 'MACD_FAST_PERIOD':...","{'MACD_FAST_PERIOD': 0.18096935958249902, 'RSI...","{'RSI': 0.188043593582728, 'MACD_FAST_PERIOD':...","{'RSI': 0.19534557352211201, 'MACD_FAST_PERIOD...","{'RSI': 0.18541451474164602, 'MACD_FAST_PERIOD...","{'RSI': 0.19050826045313102, 'MACD_FAST_PERIOD...","{'RSI': 0.213348417079255, 'MACD_FAST_PERIOD':...",...,"{'MACD_FAST_PERIOD': 0.184860323598097, 'RSI':...","{'RSI': 0.17462375247823803, 'MACD_FAST_PERIOD...","{'RSI': 0.200839749177878, 'MACD_FAST_PERIOD':...","{'RSI': 0.182915963484153, 'MACD_FAST_PERIOD':...","{'RSI': 0.23319937760571302, 'MACD_FAST_PERIOD...","{'RSI': 0.18595736051219602, 'MACD_FAST_PERIOD...","{'RSI': 0.194650016636945, 'MACD_FAST_PERIOD':...","{'RSI': 0.21053050392778402, 'MACD_FAST_PERIOD...","{'MACD_FAST_PERIOD': 0.19957204472487103, 'RSI...","{'RSI': 0.18633757632362502, 'MACD_FAST_PERIOD..."
y_test,"[buy, buy, buy, buy, sell, buy, sell, buy, sel...","[buy, buy, sell, buy, buy, buy, buy, buy, buy,...","[buy, buy, buy, buy, sell, sell, buy, buy, buy...","[buy, buy, sell, buy, sell, buy, buy, buy, buy...","[sell, buy, sell, buy, sell, buy, sell, buy, s...","[buy, buy, buy, buy, buy, buy, buy, buy, buy, ...","[buy, buy, buy, sell, buy, buy, buy, buy, buy,...","[buy, buy, buy, buy, buy, buy, buy, buy, buy, ...","[sell, buy, buy, buy, buy, buy, buy, buy, buy,...","[buy, buy, buy, buy, sell, buy, buy, buy, buy,...",...,"[sell, buy, sell, buy, sell, buy, sell, buy, s...","[buy, buy, sell, buy, sell, buy, sell, buy, se...","[sell, buy, sell, buy, sell, sell, sell, sell,...","[buy, buy, sell, sell, sell, buy, sell, buy, b...","[sell, buy, sell, sell, buy, buy, sell, buy, b...","[buy, buy, buy, buy, sell, buy, buy, buy, buy,...","[buy, buy, sell, sell, buy, buy, buy, sell, bu...","[buy, buy, sell, sell, buy, buy, buy, buy, sel...","[buy, buy, sell, buy, buy, buy, buy, buy, sell...","[sell, buy, sell, buy, sell, buy, buy, buy, se..."
y_pred,"[buy, buy, sell, buy, sell, sell, sell, buy, s...","[buy, buy, sell, buy, buy, buy, buy, buy, buy,...","[buy, buy, sell, buy, buy, sell, buy, buy, buy...","[buy, buy, sell, buy, buy, buy, buy, buy, buy,...","[sell, buy, sell, buy, sell, sell, sell, buy, ...","[buy, buy, buy, buy, buy, buy, buy, buy, buy, ...","[buy, buy, sell, sell, buy, buy, buy, buy, buy...","[buy, buy, sell, buy, buy, buy, buy, buy, buy,...","[sell, buy, sell, buy, buy, buy, buy, buy, buy...","[buy, buy, sell, buy, buy, buy, buy, buy, sell...",...,"[sell, buy, sell, buy, sell, sell, buy, sell, ...","[buy, buy, sell, buy, sell, buy, sell, buy, se...","[sell, buy, sell, buy, sell, buy, sell, sell, ...","[buy, buy, sell, buy, sell, buy, buy, buy, buy...","[sell, sell, sell, sell, buy, buy, sell, buy, ...","[buy, buy, buy, buy, buy, buy, buy, buy, buy, ...","[buy, buy, sell, sell, buy, buy, buy, sell, bu...","[buy, sell, sell, buy, buy, buy, buy, buy, sel...","[buy, sell, sell, buy, buy, buy, buy, buy, sel...","[sell, buy, sell, buy, buy, buy, buy, buy, sel..."
full_pred,"[buy, buy, buy, sell, sell, sell, sell, sell, ...","[sell, sell, sell, sell, sell, sell, sell, buy...","[sell, sell, sell, sell, sell, sell, sell, sel...","[buy, buy, buy, buy, buy, buy, buy, buy, buy, ...","[sell, sell, sell, sell, buy, buy, buy, buy, b...","[sell, sell, sell, sell, sell, sell, buy, buy,...","[buy, buy, buy, buy, buy, buy, buy, buy, buy, ...","[buy, buy, buy, buy, buy, buy, buy, buy, buy, ...","[buy, buy, buy, buy, buy, buy, buy, buy, buy, ...","[sell, sell, sell, buy, buy, sell, buy, buy, b...",...,"[sell, sell, sell, sell, sell, sell, sell, sel...","[sell, sell, sell, sell, sell, sell, sell, sel...","[sell, sell, sell, sell, sell, sell, sell, sel...","[sell, sell, sell, buy, buy, buy, sell, sell, ...","[buy, buy, se

In [11]:
y_test = rf_j['XLE'].loc['y_test']
y_pred = rf_j['XLE'].loc['y_pred']

In [19]:
balanced_accuracy_score(y_test, y_pred)

0.8823936825492344

In [20]:
# Create a Title for the Dashboard
dashboard_title = '# Machine Learning Project'

# Define a welcome text
welcome_message =  '''
    ### Welcome to Kurt, Matt and Anita's Dashboard
    '''

welcome_message_2 = '''
    In this Dasboard you will find analysis, comparing our two Machine Learning Models [Model name 1] and
    [Model name 2]. You will also see what our feature and target data sets consists of.
    '''

welcome_pane = pn.Column(
    dashboard_title,
    welcome_message,
)

welcome_pane_2 = pn.Column(
    dashboard_title,
    welcome_message_2,
)

# Create panels to structure the layout of the dashboard
tab_1 = pn.Row(
    pn.Column(ticker_widget, random_forrest_results),
    pn.Column(ticker_widget, naive_bayes_results))

tab_2 = pn.Column(ticker_widget, backtest_results)

tab_3 = pn.Column()

tab_4 = pn.Column()

tab_5 = pn.Column()

dashboard = pn.Tabs(
    ('Welcome', welcome_pane),
    ('Welcome 2', welcome_pane_2),
    ('Tab 1', tab_1),
    ('Tab 2', tab_2),
    ('Tab 3', tab_3),
    ('Tab 4', tab_4),
    ('Tab 5', tab_5)
)
dashboard.servable()

Tabs
    [0] Column
        [0] Markdown(str)
        [1] Markdown(str)
    [1] Column
        [0] Markdown(str)
        [1] Markdown(str)
    [2] Row
        [0] Column
            [0] Select(name='Ticker', options=['XTL', 'KRE', ...], value='VNQ')
            [1] ParamFunction(function)
        [1] Column
            [0] Select(name='Ticker', options=['XTL', 'KRE', ...], value='VNQ')
            [1] ParamFunction(function)
    [3] Column
        [0] Select(name='Ticker', options=['XTL', 'KRE', ...], value='VNQ')
        [1] ParamFunction(function)
    [4] Column()
    [5] Column()
    [6] Column()

In [21]:
dashboard.show()

Launching server at http://localhost:59333


In [ ]:
dashboard.servable()